In [1]:
import pandas as pd
import datetime

In [2]:
ibge = pd.read_csv('./dados/ibge_drs.csv')

#função para iterar datas
def daterange(start_date, end_date):
    for i in range(int((end_date - start_date).days)):
        yield start_date + datetime.timedelta(i)

inicio = datetime.datetime(2020,1,1)
fim = datetime.datetime.now()

for cod in ibge.iloc():
    cidade = str(cod['Código IBGE'])
    co = pd.read_csv('casos_obitos/' + cidade + '.csv')
    vacinacao = pd.read_csv('vacinacao/' + cidade + '.csv')
    isolamento = pd.read_csv('isolamento/datas_formatadas/' + cidade + '.csv')
    ocupacao = pd.read_csv('ocupacao/' + cidade + '.csv')
    
    r = pd.DataFrame()
    try:
        r = pd.read_csv('r_efetivo/cidades/' + cidade + '.csv')
    except FileNotFoundError:
        pass
    
    
    dados = { 'Município': [], 'Código IBGE': [], 'Populacao': [], 'Total 1ª Dose': [],
       'Total 2ª Dose': [], 'Total Unica': [], 'Total Doses Aplicadas': [],
       'População Vacinada dose1/População Total': [],
       'População Vacinada/População Total': [], 'data': [], 'diagnostico_covid19': [], 'obito': [], 'media_movel_casos': [],
       'media_movel_obitos': [], 'ocupacao_leitos': []}
    novo = pd.DataFrame(data=dados)
    for data in daterange(inicio, fim):
        
        iso = isolamento[isolamento['data'] == data.strftime('%Y-%m-%d')].drop(columns=['Unnamed: 0', 'DRS', 'cidade', 'cod_ibge'])
        vac = vacinacao[vacinacao['data'] == data.strftime('%Y-%m-%d')].drop(columns=['Unnamed: 0'])
        cao = co[co['data_inicio_sintomas'] == data.strftime('%Y-%m-%dT00:00:00Z')].drop(columns=['Unnamed: 0','nome_munic', 'codigo_ibge', 'data_inicio_sintomas'])
        cao['data'] = data.strftime('%Y-%m-%d')
        ocu = ocupacao[ocupacao['data'] == data.strftime('%Y-%m-%d')].drop(columns=['Unnamed: 0', 'cod_ibge', 'data'])
        ocu['data'] = data.strftime('%Y-%m-%d')
        
        soma = pd.merge(vac, cao, on="data")
        soma = pd.merge(soma, ocu, on="data")
        soma = pd.merge(soma, iso, on="data")
        
        if(not r.empty):
            ref = r[['data', 'Mean.R']]
            soma = pd.merge(soma, ref, on="data")
        else :
            soma['Mean.R'] = 0
        
        novo = novo.append(soma)
    novo = novo.reset_index().drop(columns='index')
    novo.to_csv('../Finais/' + cidade + '.csv')

#print(novo.dropna())
novo

,Município,Código IBGE,Populacao,Total 1ª Dose,Total 2ª Dose,Total Unica,Total Doses Aplicadas,População Vacinada dose1/População Total,População Vacinada/População Total,data,diagnostico_covid19,obito,media_movel_casos,media_movel_obitos,ocupacao_leitos,media_isolamento,Mean.R
0,ESTIVA GERBI,3557303.0,11407.0,206.0,0.0,0.0,206.0,1.805909,0.000000,2021-02-21,2.0,0.0,3.571429,0.000000,"67,39",52.500000,0.0
1,ESTIVA GERBI,3557303.0,11407.0,211.0,0.0,0.0,211.0,1.849741,0.000000,2021-02-22,3.0,0.0,3.285714,0.000000,"65,69",45.666667,0.0
2,ESTIVA GERBI,3557303.0,11407.0,212.0,0.0,0.0,212.0,1.858508,0.000000,2021-02-23,3.0,1.0,3.571429,0.142857,"70,37",45.833333,0.0
3,ESTIVA GERBI,3557303.0,11407.0,212.0,0.0,0.0,212.0,1.858508,0.000000,2021-02-24,3.0,0.0,3.428571,0.142857,"70,07",51.500000,0.0
4,ESTIVA GERBI,3557303.0,11407.0,214.0,0.0,0.0,214.0,1.876041,0.000000,2021-02-25,2.0,0.0,3.428571,0.142857,"71,32",45.833333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,ESTIVA GERBI,3557303.0,11407.0,4840.0,1162.0,188.0,6190.0,42.430087,11.834838,2021-07-15,0.0,0.0,1.428571,0.000000,"78,85",46.833333,0.0
145,ESTIVA GERBI,3557303.0,11407.0,4974.0,1250.0,268.0,6492.0,43.604804,13.307618,2021-07-16,0.0,0.0,1.428571,0.000000,"79,62",45.333333,0.0
146,ESTIVA GERBI,3557303.0,11407.0,5029.0,1453.0,268.0,6750.0,44.086964,15.087227,2021-07-17,1.0,0.0,1.428571,0.000000,"76,58",47.666667,0.0
147,ESTIVA GERBI,3557303.0,11407.0,5029.0,1453.0,268.0,6750.0,44.086964,15.087227,2021-07-18,0.0,0.0,0.857143,0.000000,"76,28",53.333333,0.0
